In [50]:
def separate_tag_to_key_value(tags):
    key_values = {}
    for tag in tags:
        key, value = tag.split(": ")
        key_values[key] = value
    return key_values


import wandb
import pandas as pd

wandb.login()
api = wandb.Api()
# SNRとsubsample_kのフィルタリング条件
snr_levels = [5]
subsample_ks = [2,4,8,16,32,64]
# project_names = ["cubic_spline"]
project_names = ["cubic_spline_with_time_domain"]
# project_names = ["simple_downsampling_with_time_domain"]
# project_names = ["ununiform_sampling_with_time_domain"]
# project_names = ["ununiform_sampling"]

tags_to_keep = ["snr_level"]
summary_to_keep = ["noisy_metrics", "pred_clean_metrics"]
white_result_df = pd.DataFrame()
air_conditioner_result_df = pd.DataFrame()
for project in api.projects():
    if project.name in project_names:
        runs = api.runs(f"{project.entity}/{project.name}")
        for run in runs:
            # `tags`を取得
            # print(run.config)

            config = run.config
            summary = run.summary
            tag = separate_tag_to_key_value(run.tags)

            if tag["noise_type"] == "white":
                try:
                    filtered_summary = {key: dict(summary[key]) for key in summary_to_keep if key in summary}
                    noisy_metrics_df = pd.DataFrame([filtered_summary["noisy_metrics"]])
                    pred_clean_metrics_df = pd.DataFrame([filtered_summary["pred_clean_metrics"]])
                    metrics_delta_df = pred_clean_metrics_df - noisy_metrics_df
                    metrics_delta_df["k"] = config["subsample_k"]
                    # metrics_delta_df["run_id"] = run.id
                    # metrics_delta_df["run_name"] = run.name
                    white_result_df = pd.concat([white_result_df, metrics_delta_df], ignore_index=True)
                except:
                    continue    

            elif tag["noise_type"] == "0_air_conditioner":
                filtered_summary = {key: dict(summary[key]) for key in summary_to_keep if key in summary}
                noisy_metrics_df = pd.DataFrame([filtered_summary["noisy_metrics"]])
                pred_clean_metrics_df = pd.DataFrame([filtered_summary["pred_clean_metrics"]])
                metrics_delta_df = pred_clean_metrics_df - noisy_metrics_df
                metrics_delta_df["k"] = config["subsample_k"]
                # metrics_delta_df["run_id"] = run.id
                # metrics_delta_df["run_name"] = run.name
                air_conditioner_result_df = pd.concat([air_conditioner_result_df, metrics_delta_df], ignore_index=True)

In [51]:
air_conditioner_result_df.sort_values(["sdr"], ascending=False, inplace=True)
air_conditioner_result_df
# max_01 = air_conditioner_result_df[air_conditioner_result_df["run_name"].str.contains("001")]
# max_01

,pesq,sdr,si_sdr,si_snr,snr,stoi,k
5,0.032190,3.544470,2.775240,2.775240,3.455130,-0.009707,64
3,0.026888,3.342707,2.712224,2.712224,3.403173,0.008547,16
4,0.037287,3.159879,2.561174,2.561174,3.275360,0.009079,32
9,0.018091,2.989090,2.290750,2.290750,3.086278,0.004839,16
29,0.013916,2.984980,2.196183,2.196183,2.895132,-0.002800,64
...,...,...,...,...,...,...,...
52,-0.005911,0.703409,0.505389,0.505389,1.378726,-0.012286,32
47,-0.004060,0.686227,0.289512,0.289512,1.204107,-0.011392,64
45,-0.000927,0.579597,0.409052,0.409052,1.444706,0.001599,16
63,-0.009194,0.323213,-0.625240,-0.625240,0.719451,-0.010703,16


In [52]:
# 各メトリクスの平均値を算出
white_mean = white_result_df.groupby("k").mean(numeric_only=True).reset_index()
air_con_mean = air_conditioner_result_df.groupby("k").mean(numeric_only=True).reset_index()
white_latex = white_mean.to_latex("white_mean.tex")
air_con_latex = air_con_mean.to_latex("air_con_mean.tex")   